# RAG su PDF

### con  generazione keywords per migliorare la ricerca vettoriale 

In [2]:
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_classic.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate, SystemMessagePromptTemplate

# sentence-transformers
## all-MiniLM-L6-v2
   
È un modello destinato a essere utilizzato come codificatore di frasi e paragrafi brevi (modello di Embedding). Dato un testo in input, restituisce in output un vettore che cattura l'informazione semantica contenuta nel testo di input. Il vettore della frase può essere poi utilizzato per attività di recupero di informazioni (RAG), raggruppamento o ricerca per somiglianza.   
    
**Attenzione** Il modello trasforma il testo immesso più lungo di 256 token troncandoilo a 256 token.    
*(invece Nomic-Embed gestisce sequenze da 64 a 768 token).*

In [3]:
loaders = [PyPDFLoader('data/listino.pdf')]

docs = []
for file in loaders:
    docs.extend(file.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=150)
docs = text_splitter.split_documents(docs)

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu',})

In [4]:
print(len(docs))

69


In [5]:
vectorstore = Chroma.from_documents(docs, embedding_function, persist_directory="data/chroma")

print(vectorstore._collection.count())

138


In [6]:
client = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")  # Google Gemma2 tramite LM Studio

In [7]:
vector_db = Chroma(persist_directory="data/chroma", embedding_function=embedding_function)

In [8]:
docs_retrieved = vector_db.similarity_search("lights", k=5)

for doc in docs_retrieved:
    print ("documento:", doc.metadata["source"]) 
    print (doc.page_content)
    print ("_" * 80, "\n")

documento: data/listino.pdf
smartphone. Prezzo: 120€ 
Fari LED per Fotografia - Codice: LUC009 - Kit di fari LED per studi fotografici. Prezzo: 200€ 
Lampada da Terra Moderna - Codice: LUC010 - Lampada da terra con design moderno, ideale per 
soggiorni. Prezzo: 100€
________________________________________________________________________________ 

documento: data/listino.pdf
smartphone. Prezzo: 120€ 
Fari LED per Fotografia - Codice: LUC009 - Kit di fari LED per studi fotografici. Prezzo: 200€ 
Lampada da Terra Moderna - Codice: LUC010 - Lampada da terra con design moderno, ideale per 
soggiorni. Prezzo: 100€
________________________________________________________________________________ 

documento: data/listino.pdf
Luci 
Lampada da Scrivania LED - Codice: LUC001 - Lampada da scrivania con intensità regolabile. Prezzo: 
45€ 
Luce Anello per Selfie - Codice: LUC002 - Anello luminoso per selfie e video. Prezzo: 30€ 
Proiettore Laser RGB - Codice: LUC003 - Proiettore laser per effetti l

In [9]:
retriever = vector_db.as_retriever(
    search_type="mmr",  # mmr
    search_kwargs={"k": 7}
)

In [10]:
sys = SystemMessagePromptTemplate.from_template("extract the keywords from the text and generate a list of words similar to the keywords found \nyour response should be a list of comma separated values, eg: ""foo, bar, baz""")

initial_prompt = ChatPromptTemplate(
    input_variables=["question"], 
    messages=[
        sys,
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['question'], 
                template="Question: {question}"
            )
        )
    ]
)

initial_rag_chain = initial_prompt | client | CommaSeparatedListOutputParser()

In [11]:
initial_rag_chain.invoke("do you have available rgb mouse?")

['RGB',
 'color',
 'lighting',
 'illumination',
 'hue',
 'saturation',
 'brightness',
 'mouse',
 'computer peripheral',
 'device',
 'input',
 'cursor',
 'trackpad',
 'joystick']

In [12]:
prompt = ChatPromptTemplate(
    input_variables=['context', 'question'], 
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'], 
                template="act as an ecommerce operator \nfind the following user request in the available products list provided \nreply with only a list of the products that answer the question \n\nQuestion: {question} \n\nAvailable products list: {context} \n\nAnswer:")
)])

In [13]:
def format_documents(docs):
    return "\n\n".join(doc.page_content.replace("\n", "") for doc in docs)


rag_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | prompt
    | client
    | StrOutputParser()
)

In [14]:
rag_chain.invoke(str(initial_rag_chain.invoke("do you have available rgb mouse?")))

'- Tastiera Gaming Meccanica RGB - Codice: GAM003 – Tastiera meccanica con retroilluminazione RGB personalizzabile.'

In [15]:
new_chain = (initial_prompt | client | StrOutputParser() 
                 | {"context": retriever | format_documents, "question": RunnablePassthrough()} 
                 | prompt 
                 | client 
                 | StrOutputParser()
            )

In [16]:
new_chain.invoke("do you have available rgb mouse?", config={'callbacks': [ConsoleCallbackHandler()]})  # callback per debug degli step nella chain

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "do you have available rgb mouse?"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "do you have available rgb mouse?"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: extract the keywords from the text and generate a list of words similar to the keywords found \nyour response should be a list of comma separated values, eg: foo, bar, baz\nHuman: Question: do you have available rgb mouse?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "rgb, color, RGB, lighting, illumination, computer mouse, peripheral, pointing device, input device, optical mouse, wireless mouse, gaming mous

'- Mouse Gaming ad Alta Precisione - Codice: **GAM004** – Mouse da gaming con sensore di precisione e DPI regolabile.'

In [17]:
new_chain.invoke("ci sono luci a led?")

'- Luci Lampada da Scrivania LED – Codice: LUC001  \n- Luce Anello per Selfie – Codice: LUC002  \n- Fari LED per Fotografia – Codice: LUC009  \n- Lampada da Terra Moderna (LED) – Codice: LUC010  \n- Lanterna LED Ricaricabile – Codice: LUC012  \n- Proiettore Stelle per Camerette (LED) – Codice: LUC013  \n- Luce Notturna con Sensore (LED) – Codice: LUC014'